In [ ]:
api_key = "RGAPI-11c8885f-7b92-4e0c-b084-b457e7398203"

In [ ]:
import requests


In [ ]:
#converter roman number to int (the date won't go further 5 as there is only 5 tier possible in the data)
class roman_to_int(object):
   def romanToInt(self, s):
      """
      :type s: str
      :rtype: int
      """
      roman = {'I':1,'V':5}
      i = 0
      num = 0
      while i < len(s):
         if i+1<len(s) and s[i:i+2] in roman:
            num+=roman[s[i:i+2]]
            i+=2
         else:
            num+=roman[s[i]]
            i+=1
      return num


In [226]:
## function ranking_information : take a list of summoners name and return a list of those summoners but with their rank informations (if the summoners is not found he is not added to the output list)

def ranking_information (summoner_list_name:list):
    global list_summoner_rank
    list_summoner_rank = []

    for summoner in summoner_list_name:
    
        ## retrieve two important informations (the id and puuid) for the next steps
        
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        if data_summoner is not None and "id" in data_summoner:
            id_summoner = data_summoner.get("id")
            puuid_summoner = data_summoner.get("puuid")
          
            ## with the id we can retrieve the information of the account (tier, LP, rank)  
            
            data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
        
            ## we check from the data_rank if the account has done any SOLO Q games for this season only !!
            
            presence_of_soloq_data = False
            for data_SOLOQ in data_rank:
                if data_SOLOQ['queueType'] == "RANKED_SOLO_5x5":
                    print(summoner,data_SOLOQ['tier'],data_SOLOQ['rank'],data_SOLOQ['leaguePoints'], "LP")
                    rank = roman_to_int().romanToInt(data_SOLOQ['rank'])
                    
                    ## here we save the informations for each account            
                      
                    list_summoner_rank.append((summoner,data_SOLOQ['tier'],rank,data_SOLOQ['leaguePoints']))
                    presence_of_soloq_data = True
            if not presence_of_soloq_data:
                print(f"No data / No SOLO Q for this account : {summoner}")

                ## we still keep the summoner in the final list (to avoid users thinking that the account was not checked)
                    
                list_summoner_rank.append((summoner,"none","0", "0"))
        else:
            print(f"Failed to retrieve data for summoner {summoner}")
            
#summoner_list = ["SaintLoutr","Draeknof","Haukeas Grémory","Caio73","charlos92604","7689000","Alrsay","TheK1ngPotato","Blarckful"]
summoner_list = ["SaintLoutr","TheK1ngPotato","Draeknof"]

ranking_information(summoner_list)

SaintLoutr SILVER III 54 LP
No data / No SOLO Q for this account : TheK1ngPotato
Draeknof GOLD III 25 LP


In [ ]:
import datetime
import calendar

## set the time of today

now = datetime.datetime.now()

start_of_day = datetime.datetime(
    year=now.year,
    month=now.month,
    day=now.day,
    hour=0,
    minute=0,
    second= 1,
)

end_of_day = datetime.datetime(
    year=now.year,
    month=now.month,
    day=now.day,
    hour=23,
    minute=59,
    second=59,
)

## retrieve the start of the day and the end of the day

end_of_day_epoch = str(int(end_of_day.timestamp()))
start_of_day_epoch = str(int(start_of_day.timestamp()))

print(start_of_day_epoch, end_of_day_epoch)


In [ ]:
##the function result_match take a list of summoners and return a list of the results of their SOLOQ of the day 
def result_match (summoner_list):
    
    for summoner in summoner_list:
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        puuid_summoner = data_summoner.get("puuid")
        
        ## with the id, the end of the day, and the start of the day we can check if the account has done any SOLOQ for the day
        
        matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&startTime='+ start_of_day_epoch +'&endTime='+ end_of_day_epoch +'&count=20&api_key='+api_key).json()
        print(summoner,matches_id)

        ## check if there is any match
        
        if len(matches_id) != 0:  
            for match_id in matches_id:
                
                ## check the result of the game for the summoner
                
                match_info = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+ match_id +"?api_key=" + api_key).json()
                for player in (match_info['info']['participants']):
                    if player['puuid'] == puuid_summoner:
                        if player['win'] == True:
                            print("WIN")
                        else:
                            print("LOOSE")
        else:
            print("Pas de match classé aujourd'hui")        
    list_player = []

result_match(summoner_list)

In [ ]:
#ajout des données de rank des compte dans une base de données

import sqlite3

def insert_into_leaderboard(SummonerName, Tier, Rank, LP):
  try:
    conn = sqlite3.connect('leaderboard.db')
    cur = conn.cursor()
    print("Connexion réussie à SQLite")
    sql = "INSERT INTO leaderboard (SummonerName, Tier, Rank, LP) VALUES (?, ?, ?, ?)"
    value = (SummonerName, Tier, Rank, LP)
    cur.execute(sql, value)
    conn.commit()
    print(f"Enregistrement inséré avec succès dans la table leaderboard pour le compte {SummonerName}")
    cur.close()
    conn.close()
    print("Connexion SQLite est fermée")
  except sqlite3.Error as error:
    print("Erreur lors de l'insertion dans la table leaderboard", error)

for summoner in list_summoner_rank:
    SummonerName, Tier, Rank, LP = summoner
    insert_into_leaderboard(SummonerName, Tier, Rank, LP)

In [ ]:
#suppresion de la table

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("delete from leaderboard")
   conn.commit()
   print("leaderboard supprimé")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)

In [ ]:
#sorting de la liste

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("SELECT *FROM leaderboard ORDER BY CASE WHEN tier = 'CHALLENGER' THEN 1 WHEN tier = 'GRANDMASTER' THEN 2 WHEN tier = 'MASTER' THEN 3 WHEN tier = 'DIAMOND' THEN 4 WHEN tier = 'PLATINIUM' THEN 5 WHEN tier = 'GOLD' THEN 6 WHEN tier = 'SILVER' THEN 7 WHEN tier = 'BRONZE' THEN 8 WHEN tier = 'IRON' THEN 9 ELSE 10 END,Rank asc,lp desc;")
   conn.commit()
   print("leaderboard sorted")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)